In [1]:
import numpy as np
from tensorflow import set_random_seed
from numpy.random import seed
import pandas as pd
#import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import datetime
from datetime import timedelta

Using TensorFlow backend.


In [2]:
#dataset_empty = 0
set_random_seed(1)
seed(1)

In [3]:
#Convert the train data into time series
def time_series_data(data, window, lag):
    dropnan = True
    cols, names = list(), list()

    for i in range(window, 0, -1):
        #past time series data (t-)
        cols.append(data.shift(i))
        names = names + [('%s(t-%d)' % (col, i)) for col in data.columns]
    
    #current time series data (t = 0)
    cols.append(data)
    names = names + [('%s(t)' % (col)) for col in data.columns]
    
    #future data (t + lag)
    cols.append(data.shift(-lag))
    names = names + [('%s(t+%d)' % (col, lag)) for col in data.columns]
    
    #all data
    all_data = pd.concat(cols, axis=1)
    all_data.columns = names
    
    #drops rows with NaN
    if dropnan:
        all_data.dropna(inplace=True)
    return all_data

In [4]:
dataset = pd.read_csv('interest_over_time_s1_1_3.csv', usecols = ['Store', 'Item', 'Date', 'Weekly_Demand'])
train = dataset[['Store', 'Item', 'Date', 'Weekly_Demand']]
train = train.loc[(train['Store'] == 1) & (train['Item'] == 1)]
#train = train[(train['Date'] >= '2012-01-01')]
#train = train.loc[train['Store'] == 3]

print(train.head())

   Store  Item        Date  Weekly_Demand
0      1     1  2010-02-07             99
1      1     1  2010-02-14             54
2      1     1  2010-02-21            100
3      1     1  2010-02-28              0
4      1     1  2010-03-07             51


In [5]:
#Re-arranges the train dataset to apply shift methods
train_r = train.sort_values('Date').groupby(['Item', 'Store', 'Date'], as_index=False)
train_r = train_r.agg({'Weekly_Demand':['mean']})
train_r.columns = ['Item', 'Store', 'Date', 'Weekly_Demand']
#cols_to_drop = ['Date']
#train_r.drop(cols_to_drop, axis=1, inplace=True)
train_r.head()

,Item,Store,Date,Weekly_Demand
0,1,1,2010-02-07,99
1,1,1,2010-02-14,54
2,1,1,2010-02-21,100
3,1,1,2010-02-28,0
4,1,1,2010-03-07,51


In [6]:
#weekly demand in 0-10 scale
train_r['Weekly_Demand'] = train_r['Weekly_Demand'] / 10
train_r.head()

,Item,Store,Date,Weekly_Demand
0,1,1,2010-02-07,9.9
1,1,1,2010-02-14,5.4
2,1,1,2010-02-21,10.0
3,1,1,2010-02-28,0.0
4,1,1,2010-03-07,5.1


In [7]:
#the model will use last 117 weekly demand data and 
#current timestep (7 days) to forecast next weekly demand data 12 weeks ahead

window = 117
lag = 12

series_data = time_series_data(train_r, window, lag)
series_data.head()

,Item(t-117),Store(t-117),Date(t-117),Weekly_Demand(t-117),Item(t-116),Store(t-116),Date(t-116),Weekly_Demand(t-116),Item(t-115),Store(t-115),...,Date(t-1),Weekly_Demand(t-1),Item(t),Store(t),Date(t),Weekly_Demand(t),Item(t+12),Store(t+12),Date(t+12),Weekly_Demand(t+12)
117,1.0,1.0,2010-02-07,9.9,1.0,1.0,2010-02-14,5.4,1.0,1.0,...,2012-04-29,2.1,1,1,2012-05-06,2.1,1.0,1.0,2012-07-29,2.9
118,1.0,1.0,2010-02-14,5.4,1.0,1.0,2010-02-21,10.0,1.0,1.0,...,2012-05-06,2.1,1,1,2012-05-13,0.0,1.0,1.0,2012-08-05,3.3
119,1.0,1.0,2010-02-21,10.0,1.0,1.0,2010-02-28,0.0,1.0,1.0,...,2012-05-13,0.0,1,1,2012-05-20,3.2,1.0,1.0,2012-08-12,2.1
120,1.0,1.0,2010-02-28,0.0,1.0,1.0,2010-03-07,5.1,1.0,1.0,...,2012-05-20,3.2,1,1,2012-05-27,2.1,1.0,1.0,2012-08-19,2.0
121,1.0,1.0,2010-03-07,5.1,1.0,1.0,2010-03-14,0.0,1.0,1.0,...,2012-05-27,2.1,1,1,2012-06-03,2.1,1.0,1.0,2012-08-26,2.9


In [8]:
future_dates = series_data[['Date(t+%d)' % lag]]
future_dates

,Date(t+12)
117,2012-07-29
118,2012-08-05
119,2012-08-12
120,2012-08-19
121,2012-08-26
122,2012-09-02
123,2012-09-09
124,2012-09-16
125,2012-09-23
126,2012-09-30


In [9]:
#drops last record of (t + lag)
last_record_item = 'Item(t-%d)' % window
last_record_store = 'Store(t-%d)' % window
series_data = series_data[(series_data['Item(t)'] == series_data[last_record_item])]
series_data = series_data[(series_data['Store(t)'] == series_data[last_record_store])]

#drops Item and Store columns
cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Item', 'Store', 'Date']]
for i in range(window, 0, -1):
    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Item', 'Store', 'Date']]

series_data.drop(cols_to_drop, axis=1, inplace=True)
series_data.drop(['Item(t)', 'Store(t)', 'Date(t)'], axis=1, inplace=True)

lbls_col = 'Weekly_Demand(t+%d)' % lag
lbls = series_data[lbls_col]
series_data = series_data.drop(lbls_col, axis=1)

series_data.head()

,Weekly_Demand(t-117),Weekly_Demand(t-116),Weekly_Demand(t-115),Weekly_Demand(t-114),Weekly_Demand(t-113),Weekly_Demand(t-112),Weekly_Demand(t-111),Weekly_Demand(t-110),Weekly_Demand(t-109),Weekly_Demand(t-108),...,Weekly_Demand(t-9),Weekly_Demand(t-8),Weekly_Demand(t-7),Weekly_Demand(t-6),Weekly_Demand(t-5),Weekly_Demand(t-4),Weekly_Demand(t-3),Weekly_Demand(t-2),Weekly_Demand(t-1),Weekly_Demand(t)
117,9.9,5.4,10.0,0.0,5.1,0.0,5.0,5.6,5.2,4.9,...,3.0,3.9,2.6,2.1,3.1,2.1,3.9,5.0,2.1,2.1
118,5.4,10.0,0.0,5.1,0.0,5.0,5.6,5.2,4.9,5.1,...,3.9,2.6,2.1,3.1,2.1,3.9,5.0,2.1,2.1,0.0
119,10.0,0.0,5.1,0.0,5.0,5.6,5.2,4.9,5.1,0.0,...,2.6,2.1,3.1,2.1,3.9,5.0,2.1,2.1,0.0,3.2
120,0.0,5.1,0.0,5.0,5.6,5.2,4.9,5.1,0.0,0.0,...,2.1,3.1,2.1,3.9,5.0,2.1,2.1,0.0,3.2,2.1
121,5.1,0.0,5.0,5.6,5.2,4.9,5.1,0.0,0.0,5.5,...,3.1,2.1,3.9,5.0,2.1,2.1,0.0,3.2,2.1,2.1


In [10]:
series_data.shape[0]

14

In [11]:
#train and test split

X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(series_data, lbls.values, test_size=0.1, random_state=0)
X_train = X_train_1
Y_train = Y_train_1

X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(series_data, lbls.values, test_size=0.7, random_state=0)
X_test = X_test_1
Y_test = Y_test_1

print('Train dataset:', X_train.shape)
print('Test dataset:', X_test.shape)
X_train.head()

Train dataset: (12, 118)
Test dataset: (10, 118)


,Weekly_Demand(t-117),Weekly_Demand(t-116),Weekly_Demand(t-115),Weekly_Demand(t-114),Weekly_Demand(t-113),Weekly_Demand(t-112),Weekly_Demand(t-111),Weekly_Demand(t-110),Weekly_Demand(t-109),Weekly_Demand(t-108),...,Weekly_Demand(t-9),Weekly_Demand(t-8),Weekly_Demand(t-7),Weekly_Demand(t-6),Weekly_Demand(t-5),Weekly_Demand(t-4),Weekly_Demand(t-3),Weekly_Demand(t-2),Weekly_Demand(t-1),Weekly_Demand(t)
121,5.1,0.0,5.0,5.6,5.2,4.9,5.1,0.0,0.0,5.5,...,3.1,2.1,3.9,5.0,2.1,2.1,0.0,3.2,2.1,2.1
128,0.0,0.0,5.5,5.5,5.5,5.2,0.0,0.0,5.4,0.0,...,3.2,2.1,2.1,2.9,1.9,3.1,2.2,2.2,2.9,4.1
119,10.0,0.0,5.1,0.0,5.0,5.6,5.2,4.9,5.1,0.0,...,2.6,2.1,3.1,2.1,3.9,5.0,2.1,2.1,0.0,3.2
130,5.5,5.5,5.5,5.2,0.0,0.0,5.4,0.0,5.4,5.3,...,2.1,2.9,1.9,3.1,2.2,2.2,2.9,4.1,2.9,3.3
126,4.9,5.1,0.0,0.0,5.5,5.5,5.5,5.2,0.0,0.0,...,2.1,0.0,3.2,2.1,2.1,2.9,1.9,3.1,2.2,2.2


In [12]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
print('Train dataset:', X_train_series.shape)
print('Test dataset:', X_test_series.shape)

Train dataset: (12, 118, 1)
Test dataset: (10, 118, 1)


In [13]:
#CNN-LSTM
subsequences = 2
time_steps = X_train_series.shape[1] // subsequences
X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, time_steps, 1))
X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, time_steps, 1))
print('Train dataset:', X_train_series_sub.shape)
print('Test dataset:', X_test_series_sub.shape)

Train dataset: (12, 2, 59, 1)
Test dataset: (10, 2, 59, 1)


In [14]:
epochs = 5000
batch = 32
learning_rate = 0.00000001
adam = optimizers.Adam(learning_rate)

In [16]:
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape = (None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(60, activation = 'relu'))
model_cnn_lstm.add(Dropout(0.5))
model_cnn_lstm.add(Dense(30, activation = 'relu'))
model_cnn_lstm.add(Dropout(0.5))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')
model_cnn_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, None, 59, 64)      128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 29, 64)      0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 1856)        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60)                460080    
_________________________________________________________________
dropout_3 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1830      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
__________

In [18]:
file_name = "cnn_lstm_gtrends_weights_s1_i1.hdf5"    #saves file to create checkpoints for each epoch
checkpoint = ModelCheckpoint(file_name, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

print('Please wait...')

cnn_lstm = model_cnn_lstm.fit(X_train_series_sub, Y_train, validation_data = (X_test_series_sub, Y_test), epochs = epochs, verbose = 1, callbacks = callbacks_list, batch_size = batch)

W0809 23:26:25.356479  9920 deprecation.py:323] From C:\Anaconda3\envs\tf-cpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Please wait...


W0809 23:26:25.754416  9920 deprecation_wrapper.py:119] From C:\Anaconda3\envs\tf-cpu\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 12 samples, validate on 10 samples
Epoch 1/5000
12/12 [==============================] - 1s 68ms/step - loss: 7.2249 - val_loss: 1.2816

Epoch 00001: loss improved from inf to 7.22494, saving model to cnn_lstm_gtrends_weights_s1_i1.hdf5
Epoch 2/5000
12/12 [==============================] - 0s 4ms/step - loss: 4.7446 - val_loss: 2.4224

Epoch 00002: loss improved from 7.22494 to 4.74457, saving model to cnn_lstm_gtrends_weights_s1_i1.hdf5
Epoch 3/5000
12/12 [==============================] - 0s 4ms/step - loss: 3.0167 - val_loss: 1.4581

Epoch 00003: loss improved from 4.74457 to 3.01675, saving model to cnn_lstm_gtrends_weights_s1_i1.hdf5
Epoch 4/5000
12/12 [==============================] - 0s 4ms/step - loss: 6.8570 - val_loss: 1.2631

Epoch 00004: loss did not improve from 3.01675
Epoch 5/5000
12/12 [==============================] - 0s 4ms/step - loss: 6.4673 - val_loss: 1.3132

Epoch 00005: loss did not improve from 3.01675
Epoch 6/5000
12/12 [============================

In [19]:
#load from saved file
file_name = "cnn_lstm_gtrends_weights_s1_i1.hdf5"

model_cnn_lstm.load_weights(file_name)
model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

#prediction
cnn_lstm_train_prediction = model_cnn_lstm.predict(X_train_series_sub)
cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub)

cnn_lstm_train_prediction

array([[2.7870297],
       [2.0497937],
       [2.1490927],
       [3.385149 ],
       [2.1342046],
       [3.1199765],
       [2.79609  ],
       [2.1301422],
       [2.1411006],
       [2.8702884],
       [3.5786223],
       [4.0945334]], dtype=float32)

In [20]:
print('Train Error (RMSE):', np.sqrt(mean_squared_error(Y_train, cnn_lstm_train_prediction)))
print('Test Error (RMSE):', np.sqrt(mean_squared_error(Y_test, cnn_lstm_test_prediction)))

Train Error (RMSE): 0.2059442889789402
Test Error (RMSE): 0.9994442677799859


In [ ]:
#save to csv
date_format = "%Y-%m-%d"
a = datetime.strptime(train['Date'].max(), date_format)
a = a + timedelta(7)

next_date = str(datetime.date(a))
next_sales = np.round(cnn_lstm_train_prediction[len(cnn_lstm_train_prediction) - 1], 2)

dataset = dataset.append(pd.DataFrame({'Store' : '1',
                                       'Item' : '1',
                                       'Date' : next_date,
                                       'Weekly_Sales' : next_sales}))
dataset.tail()

In [ ]:
dataset.to_csv('sales data-set.csv', index = False)